# Two-Stage OSR Runner (Colab / VS Code Colab Kernel)

Pure-Python notebook cells (no `!` or `%` magics) to avoid syntax issues in VS Code.

In [2]:
import os, sys, subprocess,torch

print("python:", sys.executable)
print("cwd:", os.getcwd())

try:
    subprocess.run(["nvidia-smi"], check=False)
except FileNotFoundError:
    print("nvidia-smi not found; checking torch CUDA ...")
try:
    print("torch:", torch.__version__)
    print("cuda available:", torch.cuda.is_available())
    if torch.cuda.is_available():
        print("gpu count:", torch.cuda.device_count())
        print("gpu name:", torch.cuda.get_device_name(0))
except Exception as e:
    print("torch check failed:", e)

python: /usr/bin/python3
cwd: /content
torch: 2.10.0+cu128
cuda available: True
gpu count: 1
gpu name: Tesla T4


In [3]:
# Optional: mount Drive if running on Colab
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print('Drive mounted')
except Exception as e:
    print('Drive mount skipped:', e)

Mounted at /content/drive
Drive mounted


In [4]:
from pathlib import Path
import os, subprocess

REPO_URL = 'https://github.com/spinelessknave8/FYP_code.git'
REPO_DIR = Path('/content/FYP-code')

if not REPO_DIR.exists():
    subprocess.check_call(['git', 'clone', REPO_URL, str(REPO_DIR)])

os.chdir(REPO_DIR)
print('repo root:', Path.cwd())
print('has config:', (Path('configs/default.yaml')).exists())
print('has src:', (Path('src')).exists())

repo root: /content/FYP-code
has config: True
has src: True


In [5]:
import importlib, subprocess, sys

def has(mod):
    try:
        importlib.import_module(mod)
        return True
    except Exception:
        return False

core = [
    ('numpy', 'numpy<2'),
    ('scipy', 'scipy>=1.10'),
    ('PIL', 'pillow>=9.5'),
    ('sklearn', 'scikit-learn>=1.2'),
    ('matplotlib', 'matplotlib>=3.7'),
    ('tqdm', 'tqdm>=4.65'),
    ('yaml', 'pyyaml>=6.0'),
]
missing = [req for mod, req in core if not has(mod)]
if missing:
    print('Installing missing core deps:', missing)
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', *missing])
else:
    print('Core deps already available')

if not has('torch') or not has('torchvision'):
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'torch>=2.0', 'torchvision>=0.15'])

import torch, torchvision
print('torch:', torch.__version__, 'torchvision:', torchvision.__version__)

Core deps already available
torch: 2.10.0+cu128 torchvision: 0.25.0+cu128


In [6]:
from pathlib import Path

sev = neu = None

# 1) Direct expected locations
direct_candidates = [
    (Path("/content/drive/MyDrive/datasets/severstal"), Path("/content/drive/MyDrive/datasets/neu")),
    (Path("/content/drive/Shareddrives/datasets/severstal"), Path("/content/drive/Shareddrives/datasets/neu")),
]
for s, n in direct_candidates:
    if s.exists() and n.exists():
        sev, neu = s, n
        break

# 2) Recursive fallback — include shortcut targets
if sev is None:
    search_roots = [
        Path("/content/drive/MyDrive"),
        Path("/content/drive/Shareddrives"),
        Path("/content/drive/.shortcut-targets-by-id"),  # ← shortcuts live here
    ]
    sev_hits, neu_hits = [], []
    for root in search_roots:
        if not root.exists():
            continue
        sev_hits.extend(root.glob("**/severstal"))
        neu_hits.extend(root.glob("**/neu"))

    sev_hits = [p for p in sev_hits if p.is_dir()]
    neu_hits = [p for p in neu_hits if p.is_dir()]

    if sev_hits and neu_hits:
        sev, neu = sev_hits[0], neu_hits[0]

print("severstal path:", sev)
print("neu path:", neu)

# 3) Debug: show what's actually inside the shortcut folder
if sev is None or neu is None:
    sc = Path("/content/drive/.shortcut-targets-by-id")
    if sc.exists():
        print("\nDebug: shortcut targets found:")
        for p in sc.iterdir():
            print(" ", p, "→", list(p.iterdir())[:5] if p.is_dir() else "")
    raise RuntimeError(
        "Datasets not found. Check shortcut targets above and set explicit paths if needed."
    )

severstal path: /content/drive/MyDrive/datasets/severstal
neu path: /content/drive/MyDrive/datasets/neu


In [10]:
import yaml
from pathlib import Path

# Build base Colab config
base = yaml.safe_load(Path("configs/default.yaml").read_text())
base["device"] = "cuda"
base["severstal"]["data_root"] = "/content/drive/MyDrive/datasets/severstal"
base["severstal"]["train_csv"] = "train.csv"
base["severstal"]["images_dir"] = "train_images"
base["neu"]["data_root"] = "/content/drive/MyDrive/datasets/neu"
base["output_dir"] = "/content/drive/MyDrive/fyp_outputs"

Path("configs/default.colab.yaml").write_text(yaml.safe_dump(base, sort_keys=False))
print("wrote configs/default.colab.yaml")

# Build split-specific Colab configs (so split runs also use Drive paths)
for split in ["a", "b", "c"]:
    split_cfg = yaml.safe_load(Path(f"configs/neu_split_{split}.yaml").read_text())
    merged = yaml.safe_load(yaml.safe_dump(base))  # deep copy
    merged.update(split_cfg)  # add known/unknown classes
    out = Path(f"configs/neu_split_{split}.colab.yaml")
    out.write_text(yaml.safe_dump(merged, sort_keys=False))
    print("wrote", out)

wrote configs/default.colab.yaml
wrote configs/neu_split_a.colab.yaml
wrote configs/neu_split_b.colab.yaml
wrote configs/neu_split_c.colab.yaml


In [ ]:
import time
import yaml
from pathlib import Path
from src.pipelines.notebook_entrypoints import run_two_stage_stage1, run_split_pipeline

t_all = time.time()
print("[0/6] Building Colab configs...")

# Base Colab config
base = yaml.safe_load(Path("configs/default.yaml").read_text())
base["device"] = "cuda"
base["severstal"]["data_root"] = "/content/drive/MyDrive/datasets/severstal"
base["severstal"]["train_csv"] = "train.csv"
base["severstal"]["images_dir"] = "train_images"
base["neu"]["data_root"] = "/content/drive/MyDrive/datasets/neu"
base["output_dir"] = "/content/drive/MyDrive/fyp_outputs"

Path("configs/default.colab.yaml").write_text(yaml.safe_dump(base, sort_keys=False))
print("  wrote configs/default.colab.yaml")

# Split Colab configs
split_colab = []
for s in ["a", "b", "c"]:
    split_cfg = yaml.safe_load(Path(f"configs/neu_split_{s}.yaml").read_text())
    merged = yaml.safe_load(yaml.safe_dump(base))  # deep copy
    merged.update(split_cfg)
    out = Path(f"configs/neu_split_{s}.colab.yaml")
    out.write_text(yaml.safe_dump(merged, sort_keys=False))
    split_colab.append(str(out))
    print("  wrote", out)

print("[1/6] Running sanity checks...")
assert Path("/content/drive/MyDrive/datasets/severstal/train.csv").exists(), "Missing Severstal train.csv"
assert Path("/content/drive/MyDrive/datasets/severstal/train_images").exists(), "Missing Severstal train_images/"
assert Path("/content/drive/MyDrive/datasets/neu").exists(), "Missing NEU dataset folder"
print("  sanity checks passed")

print("[2/6] Stage 1: PatchCore training/calibration")
t = time.time()
run_two_stage_stage1("configs/default.colab.yaml")
print(f"  stage 1 done in {time.time() - t:.1f}s")

for i, split in enumerate(split_colab, start=3):
    print(f"[{i}/6] Split pipeline: {split}")
    t = time.time()
    run_split_pipeline(split)
    print(f"  {split} done in {time.time() - t:.1f}s")

print(f"[6/6] All done in {time.time() - t_all:.1f}s")

[0/6] Building Colab configs...
  wrote configs/default.colab.yaml
  wrote configs/neu_split_a.colab.yaml
  wrote configs/neu_split_b.colab.yaml
  wrote configs/neu_split_c.colab.yaml
[1/6] Running sanity checks...
  sanity checks passed
[2/6] Stage 1: PatchCore training/calibration
[notebook] stage1 patchcore start: configs/default.colab.yaml


In [ ]:
# Metrics summary (Drive output path)
import json
from pathlib import Path

base = Path("/content/drive/MyDrive/fyp_outputs")
for split in ["split_a", "split_b", "split_c"]:
    p = base / split / "cascade" / "metrics.json"
    if not p.exists():
        print(split, "missing metrics:", p)
        continue
    m = json.loads(p.read_text())
    print(split, {
        "tpr_unknown_system": m.get("tpr_unknown_system"),
        "fpr_known_system": m.get("fpr_known_system"),
        "stage1_pass_rate_known": m.get("stage1_pass_rate_known"),
        "stage1_pass_rate_unknown": m.get("stage1_pass_rate_unknown"),
    })

# Aggregate + combined plots + display (pure Python, no ! magic)
import sys, subprocess
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pathlib import Path

base = Path("/content/drive/MyDrive/fyp_outputs")

subprocess.check_call([sys.executable, "-m", "src.pipelines.aggregate_osr", "--output_dir", str(base)])
subprocess.check_call([
    sys.executable, "-m", "src.pipelines.plot_combined_osr",
    "--output_dir", str(base),
    "--out_dir", str(base / "combined")
])

for split in ["split_a", "split_b", "split_c"]:
    for name in ["loss_curve.png", "acc_curve.png", "roc_osr.png", "hist_osr.png"]:
        p = base / split / "plots" / name
        if p.exists():
            plt.figure(figsize=(6, 3.5))
            plt.title(f"{split} - {name}")
            plt.imshow(mpimg.imread(p))
            plt.axis("off")
            plt.show()

for name in ["roc_combined.png", "mahalanobis_combined.png"]:
    p = base / "combined" / name
    if p.exists():
        plt.figure(figsize=(7, 4))
        plt.title(name)
        plt.imshow(mpimg.imread(p))
        plt.axis("off")
        plt.show()

# Stage-1 pass/leakage and system-vs-conditional visuals (Drive output path)
import json
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

base = Path("/content/drive/MyDrive/fyp_outputs")
splits = ["split_a", "split_b", "split_c"]

rows = []
for s in splits:
    p = base / s / "cascade" / "metrics.json"
    if not p.exists():
        print(f"{s}: missing {p}")
        continue
    m = json.loads(p.read_text())
    rows.append({
        "split": s,
        "stage1_pass_rate_known": m.get("stage1_pass_rate_known", np.nan),
        "stage1_pass_rate_unknown": m.get("stage1_pass_rate_unknown", np.nan),
        "stage1_leakage_rate_known": m.get("stage1_leakage_rate_known", np.nan),
        "stage1_leakage_rate_unknown": m.get("stage1_leakage_rate_unknown", np.nan),
        "tpr_unknown_system": m.get("tpr_unknown_system", np.nan),
        "fpr_known_system": m.get("fpr_known_system", np.nan),
        "tpr_unknown_conditional": m.get("tpr_unknown_conditional", np.nan),
        "fpr_known_conditional": m.get("fpr_known_conditional", np.nan),
    })

if not rows:
    raise RuntimeError("No cascade metrics found in /content/drive/MyDrive/fyp_outputs.")

x = np.arange(len(rows))
labels = [r["split"] for r in rows]
w = 0.35

plt.figure(figsize=(8, 4))
plt.bar(x - w/2, [r["stage1_pass_rate_known"] for r in rows], width=w, label="known pass rate")
plt.bar(x + w/2, [r["stage1_pass_rate_unknown"] for r in rows], width=w, label="unknown pass rate")
plt.xticks(x, labels)
plt.ylim(0, 1)
plt.ylabel("Rate")
plt.title("Stage-1 Pass Rates by Split")
plt.legend()
plt.grid(axis="y", alpha=0.25)
plt.show()

plt.figure(figsize=(8, 4))
plt.bar(x - w/2, [r["stage1_leakage_rate_known"] for r in rows], width=w, label="known leakage")
plt.bar(x + w/2, [r["stage1_leakage_rate_unknown"] for r in rows], width=w, label="unknown leakage")
plt.xticks(x, labels)
plt.ylim(0, 1)
plt.ylabel("Rate")
plt.title("Stage-1 Leakage Rates by Split")
plt.legend()
plt.grid(axis="y", alpha=0.25)
plt.show()

plt.figure(figsize=(8, 4))
plt.plot(labels, [r["tpr_unknown_system"] for r in rows], marker="o", label="TPR unknown (system)")
plt.plot(labels, [r["tpr_unknown_conditional"] for r in rows], marker="o", label="TPR unknown (conditional)")
plt.ylim(0, 1)
plt.ylabel("Rate")
plt.title("Unknown Detection TPR: System vs Conditional")
plt.legend()
plt.grid(alpha=0.25)
plt.show()

plt.figure(figsize=(8, 4))
plt.plot(labels, [r["fpr_known_system"] for r in rows], marker="o", label="FPR known (system)")
plt.plot(labels, [r["fpr_known_conditional"] for r in rows], marker="o", label="FPR known (conditional)")
plt.ylim(0, 1)
plt.ylabel("Rate")
plt.title("Known Rejection FPR: System vs Conditional")
plt.legend()
plt.grid(alpha=0.25)
plt.show()

print("Split-wise cascade summary:")
for r in rows:
    print(
        r["split"],
        {
            "pass_known": round(r["stage1_pass_rate_known"], 4),
            "pass_unknown": round(r["stage1_pass_rate_unknown"], 4),
            "leak_known": round(r["stage1_leakage_rate_known"], 4),
            "leak_unknown": round(r["stage1_leakage_rate_unknown"], 4),
            "tpr_sys": round(r["tpr_unknown_system"], 4),
            "fpr_sys": round(r["fpr_known_system"], 4),
        },
    )